In [6]:
import pandas as pd
import folium
from folium.plugins import FloatImage, TimestampedGeoJson

# Read the CSV file into a Pandas DataFrame
data = pd.read_csv('./Data.csv')

# Data from 2002; due to bug with folium and unix timestamp digits
data = data[data['YEAR'] >= 2002]

# Create a base map
m = folium.Map(location=[data['LATITUDE'].mean(), data['LONGITUDE'].mean()], zoom_start=5)

# Define a function to format the marker color based on 'Total'
def get_marker_color(total_co2):
    if total_co2 <= 1:
      color_map = 'green'
    elif total_co2 <= 5:
      color_map = 'yellow'
    elif total_co2 <= 10:
      color_map = 'orange'
    else:
      color_map = 'red'
    return color_map

# Create GeoJSON data
features = []
for index, row in data.iterrows():
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['LONGITUDE'], row['LATITUDE']]
        },
        "properties": {
            "time": row['datetime'],
            "popup": f"CO2 Measurements (metric tons per capita) of Country: {row['Total']}",
            "icon": "circle",
            "iconstyle": {
                "fillColor": get_marker_color(row['Total']),
                "color":"black",
                "weight":1,
                "fillOpacity": 0.8,
                "radius": 6
            }
        }
    }
    features.append(feature)

# Creates geojson_data object to create a FeatureCollection
geojson_data = {
    "type": "FeatureCollection",
    "features": features
}

# Create the TimestampedGeoJson layer
TimestampedGeoJson(
    geojson_data,
    period="P1M", 
    duration="P1Y",
    add_last_point=True
).add_to(m)

# Save the interactive map to an HTML file
m.save('./interactive_map_CO2.html')